In [1]:
# Standard Library Imports
import os
import pickle
import shutil
import time

In [2]:
!pip install fastapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.6 MB/s eta 0:00:00


In [3]:
# Third-party Library Imports
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
import cv2
import numpy as np
from fastapi import FastAPI, HTTPException
from fastapi.responses import HTMLResponse, RedirectResponse
from multiprocessing import Pool
from tqdm import tqdm

In [4]:
pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 8.7 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=57e9f6090b3c50aed2479f13db7f0290ba5d25dd950b1ab0389c4bcefa0576d2
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [5]:
# Deep Learning Model Imports
from deepface import DeepFace as dpf
from deepface.commons import functions

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [6]:
pip install retinaface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 12.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of retinaface to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 22.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of retinaface to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install retinaface==0.0.1, retinaface==0.0.2, retinaface==0.0.3, retinaface==0.0.4, retinaface==0.0.5, retinaface==0.0.6, retinaface==1.1.0 and retinaface==1.1.1 because these package versions have conflicting dependen

In [7]:
# Computer Vision Model Imports
from retinaface import RetinaFace

In [8]:
pip install uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [9]:
import uvicorn

In [10]:
# Visualization
import matplotlib.pyplot as plt

In [11]:
# Creating the fastapi object
app = FastAPI()

In [12]:
# Constants
FACE_MAX_WORKERS = 14
PKL_MAX_WORKERS = 12
RECORDING_OUTPUT = "Frames"
FACE_OUTPUT = "Data"
FACE_THRESHOLD = 0.99
FPS = 30
CAMERA_INDEX = 0
DIRECTORY = os.getcwd()

In [13]:
def process_image(employee):
    model_name = "VGG-Face"
    enforce_detection = False
    detector_backend = "retinaface"
    align = True
    normalization = "base"

    target_size = functions.find_target_size(model_name=model_name)

    img_objs = functions.extract_faces(img=employee, target_size=target_size, detector_backend=detector_backend, grayscale=False, enforce_detection=enforce_detection, align=align)
    representations = []
    for img_content, _, _ in img_objs:
        embedding_obj = Deep.represent(img_path=img_content, model_name=model_name, enforce_detection=enforce_detection, detector_backend="skip", align=align, normalization=normalization)
        img_representation = embedding_obj[0]["embedding"]
        instance = [employee, img_representation]
        representations.append(instance)
    return representations

In [14]:
class WebcamRecorder:
    def __init__(self, camera_index=CAMERA_INDEX):
        self.camera_index = camera_index
        self.capture = cv2.VideoCapture(self.camera_index)
        if not self.capture.isOpened():
            raise Exception("Could not open the webcam.")

    def get_frame(self):
        ret, frame = self.capture.read()
        if not ret:
            raise Exception("Could not read a frame from the webcam.")
        return frame

    def generate_filename(self, count):
        return f'Frame {count:02d}.jpg'

    def record_video(self, path, duration=10):
        frames_dir = os.path.join(path, RECORDING_OUTPUT)
        os.makedirs(frames_dir, exist_ok=True)
        start_time = time.time()
        end_time = start_time + duration
        count = -1

        while True:
            count += 1
            filename = os.path.join(frames_dir, self.generate_filename(count))
            frame = self.get_frame()
            cv2.imwrite(filename, frame)
            print(filename)
            cv2.imshow('Webcam Feed', frame)

            if cv2.waitKey(1000 // FPS) & 0xFF == ord('q') or time.time() > end_time or count > 98:
                break

        self.capture.release()
        cv2.destroyAllWindows()

In [15]:
class FaceExtractor:
    def process_frame(self, filename):
        try:
            count = os.path.split(filename)[1].split(' ')[1].split('.')[0]
            filename1 = os.path.join(os.path.join(DIRECTORY, RECORDING_OUTPUT), filename)

            faces = RetinaFace.extract_faces(filename1, threshold=FACE_THRESHOLD, model=None, allow_upscaling=True)

            if not faces:
                print(f"No faces extracted in Frame {count}.")
                return None

            path0 = os.path.join(DIRECTORY, FACE_OUTPUT)
            if not os.path.exists(path0):
                os.makedirs(path0)

            id = 1

            image_sizes = [np.prod(image.shape) for image in faces]
            largest_index = np.argmax(image_sizes)
            filename2 = f"user.{id}.{count}.jpg"
            result = os.path.join(path0, filename2)
            print(f"user.1.{count}.jpg written")
            cv2.imwrite(result, faces[largest_index])

            return count

        except Exception as e:
            print(str(e))
            return None

    async def video_to_faces_parallel(self):
        os.chdir(DIRECTORY)
        if os.path.exists(RECORDING_OUTPUT):
            files = os.listdir(os.path.join(DIRECTORY, RECORDING_OUTPUT))

            with concurrent.futures.ThreadPoolExecutor(max_workers=FACE_MAX_WORKERS) as executor:
                futures = [executor.submit(self.process_frame, filename) for filename in files]
                concurrent.futures.wait(futures)
        else:
            print("No photos folder found.")

In [16]:
class Cleanup:
    @staticmethod
    def clean():
        directory_path = os.path.join(DIRECTORY, RECORDING_OUTPUT)
        try:
            shutil.rmtree(directory_path)
            print(f'Directory "{directory_path}" and its contents have been successfully deleted.')
        except Exception as e:
            print(f'An error occurred: {e}')

In [17]:
class Deep:
    @staticmethod
    def represent(img_path, model_name="VGG-Face", enforce_detection=False, detector_backend="retinaface", align=True,
                  normalization="base"):
        resp_objs = []
        model = dpf.build_model(model_name)

        target_size = functions.find_target_size(model_name=model_name)
        if detector_backend != "skip":
            img_objs = functions.extract_faces(img=img_path, target_size=target_size, detector_backend=detector_backend,
                                               grayscale=False, enforce_detection=enforce_detection, align=align)
        else:
            if isinstance(img_path, str):
                img = functions.load_image(img_path)
            elif type(img_path).__module__ == np.__name__:
                img = img_path.copy()
            else:
                raise ValueError(f"unexpected type for img_path - {type(img_path)}")

            if len(img.shape) == 4:
                img = img[0]
            if len(img.shape) == 3:
                img = cv2.resize(img, target_size)
                img = np.expand_dims(img, axis=0)

            img_region = [0, 0, img.shape[1], img.shape[0]]
            img_objs = [(img, img_region, 0)]

        for img, region, _ in img_objs:
            img = functions.normalize_input(img=img, normalization=normalization)

            if "keras" in str(type(model)):
                embedding = model.predict(img, verbose=0)[0].tolist()
            else:
                embedding = model.predict(img)[0].tolist()

            resp_obj = {"embedding": embedding, "facial_area": region}
            resp_objs.append(resp_obj)

        return resp_objs

    def create_pkl_multiprocessing(self):

        tik = time.time()

        db_path = FACE_OUTPUT
        model_name = "VGG-Face"
        enforce_detection = False
        detector_backend = "retinaface"
        align = True
        normalization = "base"
        silent = False

        file_name = f"representations_{model_name}.pkl"
        file_name = file_name.replace("-", "_").lower()

        pkl = f"{db_path}/{file_name}"

        if os.path.exists(pkl):
            os.remove(pkl)

        employees = []

        for r, _, f in os.walk(db_path):
            for file in f:
                if ((".jpg" in file.lower()) or (".jpeg" in file.lower()) or (".png" in file.lower())):
                    exact_path = os.path.join(r, file)
                    employees.append(exact_path)

        if len(employees) == 0:
            raise ValueError("There is no image in ", db_path, " folder! Validate .jpg or .png files exist in this path.")

        representations = []

        with Pool(PKL_MAX_WORKERS) as pool:
            for batch_start in range(0, len(employees), PKL_MAX_WORKERS):
                batch_employees = employees[batch_start:batch_start + PKL_MAX_WORKERS]
                representations_list = list(tqdm(pool.imap(process_image, batch_employees), total=len(batch_employees), desc="Finding representations", disable=silent))
                representations.extend([item for sublist in representations_list for item in sublist])

        with open(pkl, "wb") as f:
            pickle.dump(representations, f)

        if not silent:
            print(f"Representations stored in {db_path}/{file_name}. Please delete this file when you add new identities in your database.")

        tok = time.time()

        print("Multiprocessing: ",str(round(tok-tik)))

    def create_pkl_nonopti(self):

        tik = time.time()

        db_path = FACE_OUTPUT
        model_name = "VGG-Face"
        enforce_detection = False
        detector_backend = "retinaface"
        align = True
        normalization = "base"
        silent = False
        target_size = functions.find_target_size(model_name=model_name)

        file_name = f"representations_{model_name}.pkl"
        file_name = file_name.replace("-", "_").lower()

        pkl = f"{db_path}/{file_name}"


        if os.path.exists(pkl):
            os.remove(pkl)


        employees = []

        for r, _, f in os.walk(db_path):
            for file in f:
                if (
                    (".jpg" in file.lower())
                    or (".jpeg" in file.lower())
                    or (".png" in file.lower())
                ):
                    exact_path = r + "/" + file
                    employees.append(exact_path)

        if len(employees) == 0:
            raise ValueError(
                "There is no image in ",
                db_path,
                " folder! Validate .jpg or .png files exist in this path.",
            )

        # ------------------------
        # find representations for db images

        representations = []

        # for employee in employees:
        pbar = tqdm(
            range(0, len(employees)),
            desc="Finding representations",
            disable=silent,
        )
        for index in pbar:
            employee = employees[index]

            img_objs = functions.extract_faces(
                img=employee,
                target_size=target_size,
                detector_backend=detector_backend,
                grayscale=False,
                enforce_detection=enforce_detection,
                align=align,
            )

            for img_content, _, _ in img_objs:
                embedding_obj = self.represent(
                    img_path=img_content,
                    model_name=model_name,
                    enforce_detection=enforce_detection,
                    detector_backend="skip",
                    align=align,
                    normalization=normalization,
                )

                img_representation = embedding_obj[0]["embedding"]

                instance = []
                instance.append(employee)
                instance.append(img_representation)
                representations.append(instance)

        # -------------------------------

        with open(f"{db_path}/{file_name}", "wb") as f:
            pickle.dump(representations, f)

        if not silent:
            print(
                f"Representations stored in {db_path}/{file_name} file."
                + "Please delete this file when you add new identities in your database."
            )

        tok = time.time()

        print("Non Optimized: ",str(round(tok-tik)))

    def create_pkl_concurrent(self):

        tik = time.time()

        db_path = FACE_OUTPUT
        model_name = "VGG-Face"
        enforce_detection = False
        detector_backend = "retinaface"
        align = True
        normalization = "base"
        silent = False
        target_size = functions.find_target_size(model_name=model_name)

        file_name = f"representations_{model_name}.pkl"
        file_name = file_name.replace("-", "_").lower()

        print(file_name)


        pkl = f"{db_path}/{file_name}"

        if os.path.exists(pkl):
            os.remove(pkl)

        print(pkl)

        # # Create a tqdm progress bar object
        # progress_bar = tqdm(total=(len(os.listdir(os.path.join(DIRECTORY,FACE_OUTPUT)))), desc="Processing", unit=" items")

        employees = []

        for r, _, f in os.walk(db_path):
            for file in f:
                if (".jpg" in file.lower()) or (".jpeg" in file.lower()) or (".png" in file.lower()):
                    exact_path = r + "/" + file
                    employees.append(exact_path)

        if len(employees) == 0:
            raise ValueError( "There is no image in ", db_path, " folder! Validate .jpg or .png files exist in this path")

        representations = []

        global count
        count = 0

        def process_employee(employee):
            img_objs = functions.extract_faces( img=employee, target_size=target_size, detector_backend=detector_backend, grayscale=False, enforce_detection=enforce_detection, align=align, )

            for img_content, _, _ in img_objs:
                embedding_obj = self.represent( img_path=img_content, model_name=model_name, enforce_detection=enforce_detection, detector_backend="skip", align=align, normalization=normalization, )

                img_representation = embedding_obj[0]["embedding"]

                instance = []
                instance.append(employee)
                instance.append(img_representation)
                representations.append(instance)
                global count
                count += 1
                print(count)
                # progress_bar.update(1)


        # Create a ThreadPoolExecutor
        max_workers = PKL_MAX_WORKERS
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            list(executor.map(process_employee, employees))

        with open(f"{db_path}/{file_name}", "wb") as f:
            pickle.dump(representations, f)

        if not silent:
            print(
                f"Representations stored in {db_path}/{file_name} file."
                + "Please delete this file when you add new identities in your database."
            )

        # # Close the progress bar
        # progress_bar.close()

        tok = time.time()

        print("Concurrent Futures: ",str(round(tok-tik)))

In [18]:
if __name__ == "__main__":
        recorder = WebcamRecorder()
        recorder.record_video(DIRECTORY)

        face_extractor = FaceExtractor()
        face_extractor.video_to_faces_parallel()

        cle = Cleanup()
        cle.clean()

        dp = Deep()
        dp.create_pkl_multiprocessing()
        dp.create_pkl_nonopti()
        dp.create_pkl_concurrent()

Exception: ignored